In [1]:
import pandas as pd
import matplotlib as plt
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 1 шаг

In [ ]:
# извлекаем данные
query_frame = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Query.csv',sep=";")
query_frame

In [ ]:
query_frame=query_frame.drop(columns=['Unnamed: 2'], axis=1)
query_frame=query_frame.drop(columns=['Unnamed: 3'], axis=1)
query_frame=query_frame[6:]
query_frame

In [ ]:
main_data = query_frame['count(event)'].values.astype(float)

### 2 шаг

In [ ]:
# строим график нашего временного ряда
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 25
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size
x = query_frame['hour_']
y = query_frame['count(event)'] 
plt.plot(x, y)
plt.show()

### 3 шаг

In [5]:
# будем предсказывать следующий 24-часовой период по прошлым периодам.
window = 24
train_data = main_data[:-window]
test_data = main_data[-window:]
print(len(train_data))
print(len(test_data))

In [ ]:
print(test_data)

In [ ]:
def window2(data, nw):
    features_set = []
    labels = []
    length_seq=len(data)
    for i in range(window, length_seq-nw):
        features_set.append(data[i-:i, 0])
        labels.append(data[i, 0])
        features_set, labels = np.array(features_set), np.array(labels)
        features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))
        return features_set
features_set = window2(train_data, window)

### 4 шаг

In [ ]:
# нормализация
scaler = MinMaxScaler(feature_range=(-1, 1))
y = y.values.reshape(-1,1)
train_data_normalized = scaler.fit_transform(y)
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)
train_data_normalized

### 5 шаг

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# создание модели

In [ ]:
model = Sequential()
# Создание слоев LSTM и Dropout
model.add(LSTM(units=50, return_sequences=True, input_shape=(features_set.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units = 1))

model.add(Dense(units = 1))

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(features_set, labels, epochs = 100, batch_size = 32)

### 6 шаг 

In [ ]:
# проверка на тестовом датасете
test_features = np.array(test_data)
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))

In [ ]:
predictions = model.predict(test_features)
predictions = scaler.inverse_transform(predictions)

### 7 шаг

In [ ]:
# визуализация предсказаний
plt.figure(figsize=(10,6))
plt.plot(apple_testing_processed, color='blue', label='Actual Apple Stock Price')
plt.plot(predictions , color='red', label='Predicted Apple Stock Price')
plt.title('Apple Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Apple Stock Price')
plt.legend()
plt.show()